In [8]:
import torch
import numpy as np
import os
import torch.nn.functional as F
import torchvision
from torchsummary import summary

from torch import nn
from PIL import Image
from torchvision import models

from model.vit import  ViT

In [21]:
net = models.resnet101(pretrained=True)
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [39]:
fc = torch.nn.Linear(in_features=2048, out_features=9, bias=True)
net.fc = fc
net
newmodel = torch.nn.Sequential(net, torch.nn.Softmax(dim=-1))

In [24]:
from torchvision import transforms
from utils.data_loader import custom_dataloader, filename_to_tensor

tf = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor()
])

train_loader, val_loader, class_names = custom_dataloader("mushrooms_test", 8)
num_classes = len(class_names)
class_names

9
9
10
11
12
9
15
18
12


['Suillus',
 'Agaricus',
 'Entoloma',
 'Amanita',
 'Russula',
 'Lactarius',
 'Cortinarius',
 'Boletus',
 'Hygrocybe']

In [25]:
sample = train_loader['img_path'][0]
img_batch = filename_to_tensor(sample, tf)
img_batch.shape

torch.Size([8, 3, 224, 224])

In [26]:
model = ViT(
    image_size = 480,
    patch_size = 32,
    num_classes = num_classes,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

In [27]:
len(train_loader['img_path'])

11

In [41]:
preds = newmodel(img_batch)
preds

tensor([[0.0885, 0.0976, 0.0958, 0.0589, 0.1220, 0.2615, 0.0906, 0.1108, 0.0744],
        [0.0544, 0.0972, 0.0979, 0.1038, 0.1292, 0.1994, 0.0973, 0.1354, 0.0853],
        [0.0849, 0.1111, 0.1151, 0.0754, 0.1295, 0.1916, 0.0975, 0.1189, 0.0760],
        [0.0767, 0.1235, 0.0944, 0.1056, 0.1558, 0.1355, 0.1255, 0.0805, 0.1025],
        [0.0693, 0.1090, 0.0852, 0.0813, 0.1357, 0.1937, 0.0966, 0.1251, 0.1040],
        [0.0898, 0.1074, 0.1462, 0.0902, 0.1361, 0.1724, 0.0636, 0.1270, 0.0674],
        [0.0900, 0.0992, 0.1351, 0.1032, 0.0987, 0.1784, 0.1049, 0.1045, 0.0859],
        [0.1247, 0.1042, 0.1005, 0.1022, 0.1033, 0.1770, 0.1010, 0.1050, 0.0821]],
       grad_fn=<SoftmaxBackward0>)

In [23]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [24]:
loss_fn = nn.CrossEntropyLoss()  

In [25]:
actual = F.one_hot(train_loader['label'][0], 4).type(torch.float32)

In [26]:
from utils.train_loop import training_loop

n_epochs = 10
transform = tf
saved_path = 'weight'

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, transform, saved_path, eval_interval=3)

KeyboardInterrupt: 

: 